In [1]:
import subprocess   
def count_sequences_in_fasta(file_path):
    with open(file_path, 'r') as file:
        return sum(1 for line in file if line.startswith('>'))
import os

# Get the current PATH
current_path = os.environ.get('PATH')

# Define the directories to add
meme_bin_dir = os.path.join(os.environ['HOME'], 'meme', 'bin')
meme_libexec_dir = os.path.join(os.environ['HOME'], 'meme', 'libexec', 'meme-5.5.5')

# Add the directories to the PATH
os.environ['PATH'] = meme_bin_dir + os.pathsep + meme_libexec_dir + os.pathsep + current_path
intergen_dir="/home/davide/Desktop/genomiChro/intergeniche_RefSeq"
motivo="/home/davide/Downloads/motivo16.meme"

import pandas as pd
df=pd.DataFrame(columns=["num_geni","num_motivi"])
df.index.name="Ceppo"
for file in os.listdir(intergen_dir):
    if file.endswith(".fasta"):
        name=file[file.find("_"):file.find("_GCF")]
        name=name.split("_")[2:]
        name="_".join(name)
        df.loc[name,"num_geni"]=count_sequences_in_fasta(os.path.join(intergen_dir,file))
        #create directory for each file in intergen_dir/fimo
        dir_name=os.path.join(intergen_dir,"fimo",file[:-6])
        if not os.path.exists(dir_name):
            os.mkdir(dir_name)

        command = f"fimo --oc {dir_name} --verbosity 1 --bgfile --nrdb-- --thresh 1.0E-4 --norc {motivo} {os.path.join(intergen_dir,file)}"
        #os.system(command)
#make this command fimo --oc . --verbosity 1 --bgfile --nrdb-- --thresh 1.0E-4 --norc motifs.meme intergeniche.fasta


In [2]:
fimo_dir="/home/davide/Desktop/genomiChro/intergeniche_RefSeq/fimo"
import os
file="best_site.narrowPeak"

for dir in os.listdir(fimo_dir):
    #name of dir from first _ to third _
    name=dir[dir.find("_"):dir.find("_GCF")]
    name=name.split("_")[2:]
    name="_".join(name)
    #create dataframe with name as row and column number of motifs
    #add name as index to df
    for file in os.listdir(os.path.join(fimo_dir,dir)):
        if file.endswith(".narrowPeak"):
            #count lines in file
            with open(os.path.join(fimo_dir,dir,file),"r") as f:
                lines = f.readlines()
            gene=[line.split()[0] for line in lines]
            num_genes_with_motif=len(set(gene))
            df.loc[name,"num_motivi"]=f"{num_genes_with_motif}({round(num_genes_with_motif/df.loc[name,'num_geni']*100,2)}%)"
            
            

In [3]:
df

,num_geni,num_motivi
Ceppo,,
CCMEE_29,5778,171(2.96%)
CCNUC1,5987,201(3.36%)
TS-821,4374,178(4.07%)
SAG_2025,7385,271(3.67%)
CCALA_043,6569,214(3.26%)
CCALA_051,5661,198(3.5%)
[FACHB-1243],6077,212(3.49%)
SAG_39.79,7547,258(3.42%)
PCC_7203,5696,207(3.63%)


In [4]:
intergen_refseq="/home/davide/Desktop/genomiChro/intergeniche_RefSeq/fimo"
genbank_dir="/home/davide/Desktop/genomiChro/annotati_Refseq"
#parse genbanks file with Biopython and see Go_process for each gene
from Bio import SeqIO
locus_to_proteinid = {}
ceppo_motivi={}
def count_GOs(ceppo,genbank_dir, fimo_best):
    with open(fimo_best, "r") as f:
        content = f.readlines()
    names = [line.split()[0] for line in content]
    GOs = []
    GOs_background = []
    go_to_genes = {}
    locus_to_proteinid[ceppo] = {}
    ceppo_motivi[ceppo]=names
    i,j=0,0
    for file2 in os.listdir(genbank_dir):
        if ceppo in file2:
            genoma = SeqIO.parse(os.path.join(genbank_dir,file2), "genbank")
            for seq in genoma:
                for record in seq.features:
                    if "locus_tag" in record.qualifiers and "protein_id" in record.qualifiers:
                        locus_to_proteinid[ceppo][record.qualifiers["locus_tag"][0]] = record.qualifiers["protein_id"][0]

                    gene=record.qualifiers.get("gene", ["None"])[0]
                    if "locus_tag"in record.qualifiers and record.qualifiers["locus_tag"][0] in names:
                        i+=1
                        go_terms = record.qualifiers.get("GO_process", ["None"])
                        for go_term in go_terms:
                            if go_term != "None":
                                GOs.append(go_term)
                                if go_term in go_to_genes:
                                    go_to_genes[go_term].append(gene)
                                else:
                                    go_to_genes[go_term] = [gene]
                    else:
                        j+=1
                        GOs_background.append([*record.qualifiers.get("GO_process", ["None"])])
    GOs = [item for item in GOs if item != "None"]
    GOs_background = [item for sublist in GOs_background for item in sublist if item != "None"]
    return GOs, GOs_background
from collections import Counter
for dir in os.listdir(intergen_refseq):
    for file in os.listdir(os.path.join(intergen_refseq,dir)):
        if file.endswith(".narrowPeak"):
            subname=dir[dir.find("_"):dir.find("_GCF")]
            name=subname.split("_")[2:]
            name="_".join(name)
            fimo_best=os.path.join(intergen_refseq,dir,file)
            GOs, GOs_background = count_GOs(subname,genbank_dir, fimo_best)

            GOs_counter = Counter(GOs)
            GOs_background_counter = Counter(GOs_background)
            for go in GOs_counter:
                df.loc[name,go] = f"{GOs_counter[go]} ({round(GOs_counter[go]/(GOs_counter[go]+GOs_background_counter[go])*100,2)}%)" 
        df=df.copy()
                
                    
        


In [5]:
df
#order columns by number of non Nan values
df = df[df.notna().sum().sort_values(ascending=False).index]
#first column is number of genes and second is number of motifs
df = df[["num_geni","num_motivi"]+[col for col in df.columns if col not in ["num_geni","num_motivi"]]]
df
#delete last row
df


,num_geni,num_motivi,GO:0006310 - DNA recombination [Evidence IEA],GO:0055085 - transmembrane transport [Evidence IEA],GO:0009432 - SOS response [Evidence IEA],GO:0006260 - DNA replication [Evidence IEA],GO:0006355 - regulation of DNA-templated transcription [Evidence IEA],GO:0006412 - translation [Evidence IEA],GO:0006281 - DNA repair [Evidence IEA],GO:0006282 - regulation of DNA repair [Evidence IEA],...,GO:0009240 - isopentenyl diphosphate biosynthetic process [Evidence IEA],GO:0006420 - arginyl-tRNA aminoacylation [Evidence IEA],GO:0006177 - GMP biosynthetic process [Evidence IEA],GO:0008360 - regulation of cell shape [Evidence IEA],GO:0006096 - glycolytic process [Evidence IEA],GO:0051068 - dihydrolipoamide metabolic process [Evidence IEA],GO:0006750 - glutathione biosynthetic process [Evidence IEA],GO:0006423 - cysteinyl-tRNA aminoacylation [Evidence IEA],GO:0051301 - cell division [Evidence IEA],GO:0042779 - tRNA 3'-trailer cleavage [Evidence IEA]
Ceppo,,,,,,,,,,,,,,,,,,,,,
CCMEE_29,5778,171(2.96%),5 (17.24%),1 (2.33%),1 (100.0%),1 (10.0%),2 (6.67%),2 (3.57%),2 (10.53%),1 (100.0%),...,NaN,NaN,NaN,NaN,1 (11.11%),1 (100.0%),1 (20.0%),1 (50.0%),1 (50.0%),NaN
CCNUC1,5987,201(3.36%),5 (15.15%),2 (2.33%),1 (50.0%),3 (20.0%),6 (5.77%),1 (1.82%),3 (13.64%),1 (100.0%),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TS-821,4374,178(4.07%),2 (10.0%),5 (6.94%),1 (50.0%),1 (6.67%),3 (4.29%),1 (1.79%),3 (15.79%),1 (100.0%),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SAG_2025,7385,271(3.67%),3 (5.08%),3 (3.12%),1 (33.33%),4 (30.77%),9 (6.77%),3 (5.36%),3 (14.29%),1 (100.0%),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CCALA_043,6569,214(3.26%),4 (6.78%),2 (1.94%),1 (33.33%),1 (7.14%),7 (5.88%),1 (1.89%),3 (13.64%),1 (100.0%),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CCALA_051,5661,198(3.5%),4 (12.12%),3 (3.49%),1 (33.33%),1 (6.67%),4 (3.96%),2 (3.57%),2 (9.09%),1 (100.0%),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
[FACHB-1243],6077,212(3.49%),2 (5.26%),4 (4.6%),1 (25.0%),2 (13.33%),4 (3.67%),1 (1.75%),3 (13.64%),1 (100.0%),...,1 (100.0%),1 (50.0%),1 (100.0%),1 (25.0%),NaN,NaN,NaN,NaN,NaN,NaN
SAG_39.79,7547,258(3.42%),8 (10.96%),2 (2.11%),1 (33.33%),2 (13.33%),8 (6.2%),1 (1.79%),3 (13.04%),1 (100.0%),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PCC_7203,5696,207(3.63%),2 (7.69%),1 (1.27%),1 (50.0%),2 (13.33%),6 (6.19%),2 (3.57%),3 (13.64%),1 (100.0%),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1 (100.0%)


In [7]:
file_kegg="/home/davide/Downloads/KEGG Mapper Reconstruction Result PCC7203.html"
file_brite="/home/davide/Downloads/KEGG Mapper Brite Result PCC7203.html"
from parseKEGG import parsekeggpathway,parsekeggbrite
def addkomapping(file_kegg,file_brite,ceppo):
    global df
    kegg_pathways=parsekeggpathway(file_kegg)
    kegg_brite=parsekeggbrite(file_brite)
    pids=[]
    for i, locus in enumerate(ceppo_motivi[ceppo]):
        if locus in locus_to_proteinid[ceppo]:
            protein_id = locus_to_proteinid[ceppo][locus]
            pids.append(protein_id)
    new_pids=[pid for pid in pids if pid in kegg_pathways.index]
    nome=ceppo.split("_")[2:]
    nome=" ".join(nome)
    print(nome)
    for col in kegg_pathways.columns:
        df.loc[nome,col]=f"{kegg_pathways.loc[new_pids,col].sum()}({round(kegg_pathways.loc[new_pids,col].sum()/kegg_pathways[col].sum()*100,2)}%)"
    df=df.copy()
    pids_brite=[pid for pid in pids if pid in kegg_brite.index]
    for col in kegg_brite.columns:
        df.loc[nome,col]=f"{kegg_brite.loc[pids_brite,col].sum()}({round(kegg_brite.loc[pids_brite,col].sum()/kegg_brite[col].sum()*100,2)}%)"
    df=df.copy()



SyntaxError: invalid syntax (parseKEGG.py, line 80)

In [62]:
file_kegg="/home/davide/Downloads/KEGGall.html"
file_brite="/home/davide/Downloads/KEGGBriteall.html"
#kegg_pathways=parsekeggpathway(file_kegg)
kegg_brite=parsekeggbrite(file_brite)
kegg_brite

ValueError: No objects to concatenate

In [20]:
df

,num_geni,num_motivi,GO:0006310 - DNA recombination [Evidence IEA],GO:0055085 - transmembrane transport [Evidence IEA],GO:0009432 - SOS response [Evidence IEA],GO:0006260 - DNA replication [Evidence IEA],GO:0006355 - regulation of DNA-templated transcription [Evidence IEA],GO:0006412 - translation [Evidence IEA],GO:0006281 - DNA repair [Evidence IEA],GO:0006282 - regulation of DNA repair [Evidence IEA],...,GO:0009240 - isopentenyl diphosphate biosynthetic process [Evidence IEA],GO:0006420 - arginyl-tRNA aminoacylation [Evidence IEA],GO:0006177 - GMP biosynthetic process [Evidence IEA],GO:0008360 - regulation of cell shape [Evidence IEA],GO:0006096 - glycolytic process [Evidence IEA],GO:0051068 - dihydrolipoamide metabolic process [Evidence IEA],GO:0006750 - glutathione biosynthetic process [Evidence IEA],GO:0006423 - cysteinyl-tRNA aminoacylation [Evidence IEA],GO:0051301 - cell division [Evidence IEA],GO:0042779 - tRNA 3'-trailer cleavage [Evidence IEA]
Ceppo,,,,,,,,,,,,,,,,,,,,,
CCMEE_29,5778,171(2.96%),5 (17.24%),1 (2.33%),1 (100.0%),1 (10.0%),2 (6.67%),2 (3.57%),2 (10.53%),1 (100.0%),...,NaN,NaN,NaN,NaN,1 (11.11%),1 (100.0%),1 (20.0%),1 (50.0%),1 (50.0%),NaN
CCNUC1,5987,201(3.36%),5 (15.15%),2 (2.33%),1 (50.0%),3 (20.0%),6 (5.77%),1 (1.82%),3 (13.64%),1 (100.0%),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TS-821,4374,178(4.07%),2 (10.0%),5 (6.94%),1 (50.0%),1 (6.67%),3 (4.29%),1 (1.79%),3 (15.79%),1 (100.0%),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SAG_2025,7385,271(3.67%),3 (5.08%),3 (3.12%),1 (33.33%),4 (30.77%),9 (6.77%),3 (5.36%),3 (14.29%),1 (100.0%),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CCALA_043,6569,214(3.26%),4 (6.78%),2 (1.94%),1 (33.33%),1 (7.14%),7 (5.88%),1 (1.89%),3 (13.64%),1 (100.0%),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CCALA_051,5661,198(3.5%),4 (12.12%),3 (3.49%),1 (33.33%),1 (6.67%),4 (3.96%),2 (3.57%),2 (9.09%),1 (100.0%),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
[FACHB-1243],6077,212(3.49%),2 (5.26%),4 (4.6%),1 (25.0%),2 (13.33%),4 (3.67%),1 (1.75%),3 (13.64%),1 (100.0%),...,1 (100.0%),1 (50.0%),1 (100.0%),1 (25.0%),NaN,NaN,NaN,NaN,NaN,NaN
SAG_39.79,7547,258(3.42%),8 (10.96%),2 (2.11%),1 (33.33%),2 (13.33%),8 (6.2%),1 (1.79%),3 (13.04%),1 (100.0%),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PCC_7203,5696,207(3.63%),2 (7.69%),1 (1.27%),1 (50.0%),2 (13.33%),6 (6.19%),2 (3.57%),3 (13.64%),1 (100.0%),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1 (100.0%)


In [17]:
df.to_csv("/home/davide/Desktop/tabella1.csv")

In [6]:
import pandas as pd
import os
eggnog_dir="/home/davide/Desktop/genomiChro/eggnog"
df_eggnog=pd.DataFrame()
x="\t"
for file in os.listdir(eggnog_dir):
    if file.endswith(".annotations"):
        eggnog=os.path.join(eggnog_dir,file)
        eggnog_df = pd.read_csv(eggnog, sep='\t', header=4,skipfooter=3, engine='python')
        eggnog_df = eggnog_df[['#query' ,'KEGG_ko']]
        
        #rename GOs to GOs_eggnog and KEGG_ko to ko_eggnog
        eggnog_df = eggnog_df.rename(columns={ 'KEGG_ko': 'ko_eggnog', '#query': 'protein_id'})
        #set index to protein_id
        eggnog_df = eggnog_df.set_index('protein_id')

        #concat all df_eggnog but separate by #name row in a csv
        name=file[file.find("_")+1:file.find("(")-1]
        name=name.split("_")[2:]
        name="_".join(name)
        with open('/home/davide/Desktop/allkoeggnog.txt', 'a') as f:
            f.write(f"#{name}\n")
            for index,row in eggnog_df.iterrows():
                if "," in row['ko_eggnog']:
                    f.write(f"{index}{x}{row['ko_eggnog'][3:str(row['ko_eggnog']).find(',')]}\n")
                else:
                    f.write(f"{index}{x}{row['ko_eggnog'][3:]}\n")
                    print(row['ko_eggnog'],row['ko_eggnog'][3:])
        #eggnog_df.to_csv('/home/davide/PycharmProjects/TESI2/allko.txt', mode='a', header=False, sep=x)
        x+="\t"
        
    

ko:K02518 K02518
ko:K02110 K02110
ko:K02718 K02718
ko:K02691 K02691
- 
ko:K13628 K13628
ko:K02710 K02710
- 
- 
- 
- 
- 
ko:K03075 K03075
ko:K02871 K02871
ko:K02879 K02879
ko:K02948 K02948
ko:K02952 K02952
ko:K02919 K02919
ko:K00939 K00939
ko:K03076 K03076
ko:K02988 K02988
ko:K02881 K02881
ko:K02931 K02931
ko:K02874 K02874
ko:K02961 K02961
ko:K02904 K02904
ko:K02878 K02878
ko:K02890 K02890
ko:K02965 K02965
ko:K02892 K02892
ko:K02914 K02914
- 
ko:K02095 K02095
ko:K02470 K02470
ko:K09710 K09710
ko:K09940 K09940
ko:K02500 K02500
- 
ko:K01829 K01829
- 
- 
ko:K21561 K21561
- 
- 
ko:K04077 K04077
- 
- 
ko:K03841 K03841
- 
- 
ko:K08999 K08999
ko:K17892 K17892
ko:K08697 K08697
ko:K08696 K08696
ko:K08696 K08696
ko:K02696 K02696
ko:K02095 K02095
- 
- 
- 
- 
- 
ko:K09775 K09775
- 
- 
- 
- 
ko:K02066 K02066
- 
ko:K02954 K02954
- 
- 
- 
ko:K09747 K09747
- 
- 
ko:K02990 K02990
- 
ko:K02706 K02706
ko:K03561 K03561
ko:K03676 K03676
ko:K02078 K02078
ko:K02097 K02097
ko:K02112 K02112
ko:K19032 K19032
ko:

In [9]:
lines = [
    "Chroococcidiopsis_cubana_CCALA_043_GCF_003003835_intergen.fasta has 29566 lines",
    "Chroococcidiopsis_cubana_SAG_39.79_GCF_034438355.1_intergen.fasta has 34372 lines",
    "Chroococcidiopsis_sp._CCALA_051_GCF_003015105_intergen.fasta has 25651 lines",
    "Chroococcidiopsis_sp._CCMEE_29_GCF_023558375_intergen.fasta has 26123 lines",
    "Chroococcidiopsis_sp._CCNUC1_GCF_023546805_intergen.fasta has 27039 lines",
    "Chroococcidiopsis_sp._[FACHB-1243]_GCF_014696895_intergen.fasta has 27576 lines",
    "Chroococcidiopsis_sp._SAG_2025_GCF_032860985_intergen.fasta has 33576 lines",
    "Chroococcidiopsis_sp._TS-821_GCF_002939305_intergen.fasta has 20017 lines",
    "Chroococcidiopsis_thermalis_PCC_7203_GCF_000317125_intergen.fasta has 25886 lines"
]

# Extract the number of lines from each string and sum them
total_lines = sum(int(line.split()[-2]) for line in lines)

print(total_lines)

249806


In [10]:
#extract last line of file
file="/home/davide/Desktop/genomiChro/intergeniche_RefSeq/fimo/Chroococcidiopsis_cubana_CCALA_043_GCF_003003835_intergen/fimo.tsv"
with open(file, "r") as f:
    lines = f.readlines()
    last_line = lines[-1]
    print(last_line)

# fimo --oc /home/davide/Desktop/genomiChro/intergeniche_RefSeq/fimo/Chroococcidiopsis_cubana_CCALA_043_GCF_003003835_intergen --verbosity 1 --bgfile --nrdb-- --thresh 1.0E-4 --norc /home/davide/Downloads/motivo16.meme /home/davide/Desktop/genomiChro/intergeniche_RefSeq/Chroococcidiopsis_cubana_CCALA_043_GCF_003003835_intergen.fasta


In [ ]:
# fimo --oc /home/davide/Desktop/genomiChro/intergeniche_RefSeq/fimo/--verbosity 1 --bgfile --nrdb-- --thresh 1.0E-4 --norc /home/davide/Downloads/motivo16.meme /home/davide/Desktop/genomiChro/intergeniche_RefSeq/joined.fa

In [7]:
locustopid = {k: v for d in locus_to_proteinid.values() for k, v in d.items()}
file="/home/davide/Desktop/genomiChro/intergeniche_RefSeq/fimoall/joined/best_site.narrowPeak"

names=[locustopid.get(line.split()[0],'') for line in open(file).readlines()]
names=[name for name in names if name!='']


In [25]:
import os
from bs4 import BeautifulSoup
from pprint import pprint
file=("/home/davide/Downloads/KEGGall.html")
soup=BeautifulSoup(open(file), 'html.parser')
li_tags = soup.find_all("li")
#delete first elements unitl one has Metabolic patwhays in text
for i, li in enumerate(li_tags):
    if "Metabolic pathways" in str(li):
        break
li_tags=li_tags[i:]

pathways = {}
#now iterate over the li tags and find the dd tags inside them
for li in li_tags:
    dd_tags = li.find_all("dd")
    pathway=str(li.contents[1])[:-1].strip()
    diz = {}
    for dd in dd_tags:
        contents = dd.contents
        contents=[c for c in contents if c.name != 'br']
        contents=[str(content).replace('\xa0', '') for content in contents]
        contents=[str(c) for c in contents]
        for i, e in enumerate(contents):
            e=e.split(":")
            pids=e[1].split(", ")

            key=e[0]
            if key not in diz:
                diz[key]=pids
            elif pids!=['']:
                diz[key].extend(pids)
    pathways[pathway]=diz

#now for every pathway a counter of the proteins found in every ceppo
for pathway in pathways:
    diz=pathways[pathway]
    for key in diz:
        diz[key]=len(set(diz[key]))

pprint(pathways)

{'2-Oxocarboxylic acid metabolism': {'CCALA_043': 33,
                                     'CCALA_051': 33,
                                     'CCMEE_29': 36,
                                     'CCNUC1': 33,
                                     'PCC_7203': 33,
                                     'SAG_2025': 35,
                                     'SAG_39.79': 39,
                                     'TS-821': 30,
                                     '[FACHB-1243]': 31},
 'ABC transporters': {'CCALA_043': 141,
                      'CCALA_051': 128,
                      'CCMEE_29': 110,
                      'CCNUC1': 125,
                      'PCC_7203': 123,
                      'SAG_2025': 137,
                      'SAG_39.79': 149,
                      'TS-821': 115,
                      '[FACHB-1243]': 130},
 'AMPK signaling pathway': {'CCALA_043': 5,
                            'CCALA_051': 3,
                            'CCMEE_29': 3,
                            'CCNU

In [26]:
from copy import deepcopy
pathways_motifs={}
for li in li_tags:
    dd_tags = li.find_all("dd")
    pathway=str(li.contents[1])[:-1].strip()
    diz = {}
    for dd in dd_tags:
        contents = dd.contents
        contents=[c for c in contents if c.name != 'br']
        contents=[str(content).replace('\xa0', '') for content in contents]
        contents=[str(c) for c in contents]
        for i, e in enumerate(contents):
            e=e.split(":")
            pids=e[1].split(", ")
            pids=[pid for pid in pids if pid in names]
            key=e[0]
            if key not in diz:
                diz[key]=pids
            elif pids!=[''] and len(pids)!=0:
                diz[key].extend(pids)
    pathways_motifs[pathway]=diz
pathways_genes=deepcopy(pathways_motifs)
#now for every pathway a counter of the proteins found in every ceppo
for pathway in pathways_motifs:
    diz=pathways_motifs[pathway]
    for key in diz:
        diz[key]=len(set(diz[key]))

In [27]:
pathways_genes

{'Metabolic pathways': {'SAG_2025': ['WP_317106986.1',
   'WP_317108507.1',
   'WP_317108429.1',
   'WP_317107420.1',
   'WP_317104870.1',
   'WP_317104666.1',
   'WP_317110507.1',
   'WP_317107894.1',
   'WP_317106010.1',
   'WP_317105825.1',
   'WP_317105417.1',
   'WP_317109189.1',
   'WP_015156256.1',
   'WP_015153888.1',
   'WP_317105824.1',
   'WP_317106964.1',
   'WP_317104631.1',
   'WP_015156433.1',
   'WP_106218380.1',
   'WP_317108570.1',
   'WP_317109341.1',
   'WP_015157307.1',
   'WP_317110448.1'],
  'PCC_7203': ['WP_015153605.1',
   'WP_015153221.1',
   'WP_015157227.1',
   'WP_015154222.1',
   'WP_015156461.1',
   'WP_219336057.1',
   'WP_015156874.1',
   'WP_219336029.1',
   'WP_015153731.1',
   'WP_041463457.1',
   'WP_015153485.1',
   'WP_015156635.1',
   'WP_015156256.1',
   'WP_015153888.1',
   'WP_015153219.1',
   'WP_015153732.1',
   'WP_281168619.1',
   'WP_015152349.1',
   'WP_015152167.1',
   'WP_015156433.1',
   'WP_015155865.1',
   'WP_015156800.1',
   'WP_0

In [10]:
df_ko=pd.DataFrame(pathways_motifs)
df_ko
#order columns for sum of values
df_ko = df_ko[df_ko.sum().sort_values(ascending=False).index]
df_ko

,Metabolic pathways,Biosynthesis of secondary metabolites,Two-component system,Biosynthesis of cofactors,Mismatch repair,Homologous recombination,Microbial metabolism in diverse environments,Oxidative phosphorylation,DNA replication,Biosynthesis of amino acids,...,NF-kappa B signaling pathway,TNF signaling pathway,HIF-1 signaling pathway,Calcium signaling pathway,Phosphatidylinositol signaling system,Phospholipase D signaling pathway,PI3K-Akt signaling pathway,AMPK signaling pathway,Neuroactive ligand-receptor interaction,Steroid degradation
SAG_2025,23,12,7,6,7,5,5,3,4,4,...,0,0,0,0,0,0,0,0,0,0
PCC_7203,27,12,6,5,4,3,3,4,3,2,...,0,0,0,0,0,0,0,0,0,0
CCNUC1,27,14,5,6,5,5,3,3,4,2,...,0,0,0,0,0,0,0,0,0,0
CCMEE_29,17,6,3,4,3,4,4,0,2,2,...,0,0,0,0,0,0,0,0,0,0
TS-821,26,13,3,8,4,4,9,3,3,5,...,0,0,0,0,0,0,0,0,0,0
SAG_39.79,27,12,7,7,4,4,4,4,3,2,...,0,0,0,0,0,0,0,0,0,0
CCALA_043,25,11,8,6,4,5,2,4,3,2,...,0,0,0,0,0,0,0,0,0,0
CCALA_051,24,11,8,7,4,3,2,4,2,2,...,0,0,0,0,0,0,0,0,0,0
[FACHB-1243],25,13,7,5,5,3,1,3,3,2,...,0,0,0,0,0,0,0,0,0,0


In [11]:

#delete columns with only 0
df = df.loc[:, (df != 0).any(axis=0)]
df

,num_geni,num_motivi,GO:0006310 - DNA recombination [Evidence IEA],GO:0055085 - transmembrane transport [Evidence IEA],GO:0009432 - SOS response [Evidence IEA],GO:0006260 - DNA replication [Evidence IEA],GO:0006355 - regulation of DNA-templated transcription [Evidence IEA],GO:0006412 - translation [Evidence IEA],GO:0006281 - DNA repair [Evidence IEA],GO:0006282 - regulation of DNA repair [Evidence IEA],...,GO:0009240 - isopentenyl diphosphate biosynthetic process [Evidence IEA],GO:0006420 - arginyl-tRNA aminoacylation [Evidence IEA],GO:0006177 - GMP biosynthetic process [Evidence IEA],GO:0008360 - regulation of cell shape [Evidence IEA],GO:0006096 - glycolytic process [Evidence IEA],GO:0051068 - dihydrolipoamide metabolic process [Evidence IEA],GO:0006750 - glutathione biosynthetic process [Evidence IEA],GO:0006423 - cysteinyl-tRNA aminoacylation [Evidence IEA],GO:0051301 - cell division [Evidence IEA],GO:0042779 - tRNA 3'-trailer cleavage [Evidence IEA]
Ceppo,,,,,,,,,,,,,,,,,,,,,
CCMEE_29,5778,171(2.96%),5 (17.24%),1 (2.33%),1 (100.0%),1 (10.0%),2 (6.67%),2 (3.57%),2 (10.53%),1 (100.0%),...,NaN,NaN,NaN,NaN,1 (11.11%),1 (100.0%),1 (20.0%),1 (50.0%),1 (50.0%),NaN
CCNUC1,5987,201(3.36%),5 (15.15%),2 (2.33%),1 (50.0%),3 (20.0%),6 (5.77%),1 (1.82%),3 (13.64%),1 (100.0%),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TS-821,4374,178(4.07%),2 (10.0%),5 (6.94%),1 (50.0%),1 (6.67%),3 (4.29%),1 (1.79%),3 (15.79%),1 (100.0%),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SAG_2025,7385,271(3.67%),3 (5.08%),3 (3.12%),1 (33.33%),4 (30.77%),9 (6.77%),3 (5.36%),3 (14.29%),1 (100.0%),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CCALA_043,6569,214(3.26%),4 (6.78%),2 (1.94%),1 (33.33%),1 (7.14%),7 (5.88%),1 (1.89%),3 (13.64%),1 (100.0%),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CCALA_051,5661,198(3.5%),4 (12.12%),3 (3.49%),1 (33.33%),1 (6.67%),4 (3.96%),2 (3.57%),2 (9.09%),1 (100.0%),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
[FACHB-1243],6077,212(3.49%),2 (5.26%),4 (4.6%),1 (25.0%),2 (13.33%),4 (3.67%),1 (1.75%),3 (13.64%),1 (100.0%),...,1 (100.0%),1 (50.0%),1 (100.0%),1 (25.0%),NaN,NaN,NaN,NaN,NaN,NaN
SAG_39.79,7547,258(3.42%),8 (10.96%),2 (2.11%),1 (33.33%),2 (13.33%),8 (6.2%),1 (1.79%),3 (13.04%),1 (100.0%),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PCC_7203,5696,207(3.63%),2 (7.69%),1 (1.27%),1 (50.0%),2 (13.33%),6 (6.19%),2 (3.57%),3 (13.64%),1 (100.0%),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1 (100.0%)


In [12]:
file=("/home/davide/Downloads/KEGGBrite.html")
soup=BeautifulSoup(open(file), 'html.parser')
li_tags = soup.find_all("li")
for i, li in enumerate(li_tags):
    if "Enzymes" in str(li):
        break
li_tags = li_tags[i:]

brites = {}
#now iterate over the li tags and find the dd tags inside them
for li in li_tags:
    dd_tags = li.find_all("dd")
    brite=li.contents[0].get_text()+" - "+li.contents[1].get_text()[:-1].strip()
    diz = {}
    for dd in dd_tags:
        contents = dd.contents
        contents=[c for c in contents if c.name != 'br']
        contents=[str(content).replace('\xa0', '') for content in contents]
        contents=[str(c) for c in contents]
        for i, e in enumerate(contents):
            e=e.split(":")
            pids=e[1].split(", ")

            key=e[0]
            if key not in diz:
                diz[key]=pids
            elif pids!=['']:
                diz[key].extend(pids)
    brites[brite]=diz

for brite in brites:
    diz=brites[brite]
    for key in diz:
        diz[key]=len(set(diz[key]))

pprint(brites)

{'ko00194 - Photosynthesis proteins': {'CCALA_043': 69,
                                       'CCALA_051': 72,
                                       'CCMEE_29': 69,
                                       'CCNUC1': 73,
                                       'PCC_7203': 73,
                                       'SAG_2025': 73,
                                       'SAG_39.79': 71,
                                       'TS-821': 73,
                                       '[FACHB-1243]': 73},
 'ko00199 - Cytochrome P450': {'CCALA_043': 1,
                               'CCALA_051': 1,
                               'CCMEE_29': 1,
                               'CCNUC1': 1,
                               'PCC_7203': 1,
                               'SAG_2025': 1,
                               'SAG_39.79': 1,
                               'TS-821': 1,
                               '[FACHB-1243]': 1},
 'ko00536 - Glycosaminoglycan binding proteins': {'CCALA_043': 2,
                 

In [13]:
from copy import deepcopy
brites_motifs={}
for li in li_tags:
    dd_tags = li.find_all("dd")
    brite=li.contents[0].get_text()+" - "+li.contents[1].get_text()[:-1].strip()
    diz = {}
    for dd in dd_tags:
        contents = dd.contents
        contents=[c for c in contents if c.name != 'br']
        contents=[str(content).replace('\xa0', '') for content in contents]
        contents=[str(c) for c in contents]
        for i, e in enumerate(contents):
            e=e.split(":")
            pids=e[1].split(", ")
            pids=[pid for pid in pids if pid in names]
            key=e[0]
            if key not in diz:
                diz[key]=pids
            elif pids!=[''] and len(pids)!=0:
                diz[key].extend(pids)
    brites_motifs[brite]=diz
    
brites_genes=deepcopy(brites_motifs)
print(brites_genes)
for brite in brites_motifs:
    diz=brites_motifs[brite]
    for key in diz:
        diz[key]=len(set(diz[key]))


{'ko01000 - Enzymes': {'SAG_2025': ['WP_317105586.1', 'WP_317108507.1', 'WP_317104870.1', 'WP_317104666.1', 'WP_317110507.1', 'WP_039714178.1', 'WP_317105825.1', 'WP_317105417.1', 'WP_317109189.1', 'WP_317106651.1', 'WP_015152415.1', 'WP_317105752.1', 'WP_317106964.1', 'WP_015156433.1', 'WP_317107695.1', 'WP_317110020.1', 'WP_317105483.1', 'WP_317109341.1', 'WP_015157307.1', 'WP_317110448.1', 'WP_317106042.1'], 'PCC_7203': ['WP_015153605.1', 'WP_015156998.1', 'WP_219336057.1', 'WP_015154828.1', 'WP_015155906.1', 'WP_015153731.1', 'WP_015153485.1', 'WP_015152861.1', 'WP_015156635.1', 'WP_219336152.1', 'WP_015152415.1', 'WP_015152349.1', 'WP_015156433.1', 'WP_015155865.1', 'WP_015155403.1', 'WP_015152734.1', 'WP_015155069.1', 'WP_015154962.1', 'WP_015156800.1', 'WP_015154849.1', 'WP_015157307.1', 'WP_015153048.1', 'WP_015153991.1'], 'CCNUC1': ['WP_015153605.1', 'WP_106547394.1', 'WP_250015077.1', 'WP_250017084.1', 'WP_015155906.1', 'WP_250012819.1', 'WP_250016307.1', 'WP_015153485.1', 'W

In [105]:
brites_genes

{'ko01000 - Enzymes': {'SAG_2025': ['WP_317105586.1',
   'WP_317108507.1',
   'WP_317104870.1',
   'WP_317104666.1',
   'WP_317110507.1',
   'WP_039714178.1',
   'WP_317105825.1',
   'WP_317105417.1',
   'WP_317109189.1',
   'WP_317106651.1',
   'WP_015152415.1',
   'WP_317105752.1',
   'WP_317106964.1',
   'WP_015156433.1',
   'WP_317107695.1',
   'WP_317110020.1',
   'WP_317105483.1',
   'WP_317109341.1',
   'WP_015157307.1',
   'WP_317110448.1',
   'WP_317106042.1'],
  'PCC_7203': ['WP_015153605.1',
   'WP_015156998.1',
   'WP_219336057.1',
   'WP_015154828.1',
   'WP_015155906.1',
   'WP_015153731.1',
   'WP_015153485.1',
   'WP_015152861.1',
   'WP_015156635.1',
   'WP_219336152.1',
   'WP_015152415.1',
   'WP_015152349.1',
   'WP_015156433.1',
   'WP_015155865.1',
   'WP_015155403.1',
   'WP_015152734.1',
   'WP_015155069.1',
   'WP_015154962.1',
   'WP_015156800.1',
   'WP_015154849.1',
   'WP_015157307.1',
   'WP_015153048.1',
   'WP_015153991.1'],
  'CCNUC1': ['WP_015153605.1'

{'ko01000 - Enzymes': {'SAG_2025': 21,
  'PCC_7203': 23,
  'CCNUC1': 26,
  'CCMEE_29': 14,
  'TS-821': 29,
  'SAG_39.79': 23,
  'CCALA_043': 21,
  'CCALA_051': 22,
  '[FACHB-1243]': 23},
 'ko01001 - Protein kinases': {'SAG_2025': 2,
  'PCC_7203': 1,
  'CCNUC1': 0,
  'CCMEE_29': 0,
  'TS-821': 0,
  'SAG_39.79': 1,
  'CCALA_043': 1,
  'CCALA_051': 1,
  '[FACHB-1243]': 0},
 'ko01009 - Protein phosphatases and associated proteins': {'SAG_2025': 0,
  'PCC_7203': 0,
  'CCNUC1': 0,
  'CCMEE_29': 0,
  'TS-821': 0,
  'SAG_39.79': 0,
  'CCALA_043': 0,
  'CCALA_051': 0,
  '[FACHB-1243]': 0},
 'ko01002 - Peptidases and inhibitors': {'SAG_2025': 1,
  'PCC_7203': 1,
  'CCNUC1': 3,
  'CCMEE_29': 2,
  'TS-821': 3,
  'SAG_39.79': 1,
  'CCALA_043': 1,
  'CCALA_051': 1,
  '[FACHB-1243]': 3},
 'ko01003 - Glycosyltransferases': {'SAG_2025': 0,
  'PCC_7203': 0,
  'CCNUC1': 0,
  'CCMEE_29': 0,
  'TS-821': 0,
  'SAG_39.79': 0,
  'CCALA_043': 0,
  'CCALA_051': 0,
  '[FACHB-1243]': 0},
 'ko01005 - Lipopolysacch

In [54]:
#create dataframe with brites_motifs
df_brite=pd.DataFrame(brites_motifs)
df_brite
#delete columns with only 0
df_brite = df_brite.loc[:, (df_brite != 0).any(axis=0)]
df_brite

,ko01000 - Enzymes,ko01001 - Protein kinases,ko01002 - Peptidases and inhibitors,ko01011 - Peptidoglycan biosynthesis and degradation proteins,ko01008 - Polyketide biosynthesis proteins,ko01006 - Prenyltransferases,ko01007 - Amino acid related enzymes,ko00194 - Photosynthesis proteins,ko03000 - Transcription factors,ko03019 - Messenger RNA biogenesis,...,ko03036 - Chromosome and associated proteins,ko03400 - DNA repair and recombination proteins,ko03029 - Mitochondrial biogenesis,ko02000 - Transporters,ko02044 - Secretion system,ko02022 - Two-component system,ko02035 - Bacterial motility proteins,ko04147 - Exosome,ko02048 - Prokaryotic defense system,ko01504 - Antimicrobial resistance genes
SAG_2025,21,2,1,0,1,1,1,0,1,1,...,1,6,0,3,0,3,1,0,0,0
PCC_7203,23,1,1,1,0,1,1,1,3,1,...,0,6,0,3,0,2,0,0,0,0
CCNUC1,26,0,3,1,0,1,1,1,3,0,...,1,8,0,3,0,1,0,0,1,0
CCMEE_29,14,0,2,1,0,0,1,2,0,0,...,0,8,0,2,0,1,0,0,1,0
TS-821,29,0,3,0,1,0,2,0,0,2,...,0,7,1,5,1,0,0,1,1,2
SAG_39.79,23,1,1,1,0,1,1,1,1,1,...,0,7,0,3,0,2,0,0,1,0
CCALA_043,21,1,1,1,0,1,1,1,1,1,...,0,6,0,3,0,2,0,0,1,0
CCALA_051,22,1,1,0,0,1,1,2,2,1,...,0,7,0,3,0,2,0,0,1,0
[FACHB-1243],23,0,3,0,1,1,1,1,1,1,...,1,8,1,3,0,1,0,0,1,0


In [55]:
#order columns for sum of values
df_brite = df_brite[df_brite.sum().sort_values(ascending=False).index]
#join df with df_brite
df=df.join(df_brite)
df

,num_geni,num_motivi,GO:0006310 - DNA recombination [Evidence IEA],GO:0055085 - transmembrane transport [Evidence IEA],GO:0009432 - SOS response [Evidence IEA],GO:0006260 - DNA replication [Evidence IEA],GO:0006355 - regulation of DNA-templated transcription [Evidence IEA],GO:0006412 - translation [Evidence IEA],GO:0006281 - DNA repair [Evidence IEA],GO:0006282 - regulation of DNA repair [Evidence IEA],...,ko01001 - Protein kinases,ko01011 - Peptidoglycan biosynthesis and degradation proteins,ko01008 - Polyketide biosynthesis proteins,ko03036 - Chromosome and associated proteins,ko03029 - Mitochondrial biogenesis,ko01504 - Antimicrobial resistance genes,ko02044 - Secretion system,ko02035 - Bacterial motility proteins,ko04147 - Exosome,ko03110 - Chaperones and folding catalysts
Ceppo,,,,,,,,,,,,,,,,,,,,,
CCMEE_29,5778,171(2.96%),5 (17.24%),1 (2.33%),1 (100.0%),1 (10.0%),2 (6.67%),2 (3.57%),2 (10.53%),1 (100.0%),...,0,1,0,0,0,0,0,0,0,0
CCNUC1,5987,201(3.36%),5 (15.15%),2 (2.33%),1 (50.0%),3 (20.0%),6 (5.77%),1 (1.82%),3 (13.64%),1 (100.0%),...,0,1,0,1,0,0,0,0,0,0
TS-821,4374,178(4.07%),2 (10.0%),5 (6.94%),1 (50.0%),1 (6.67%),3 (4.29%),1 (1.79%),3 (15.79%),1 (100.0%),...,0,0,1,0,1,2,1,0,1,1
SAG_2025,7385,271(3.67%),3 (5.08%),3 (3.12%),1 (33.33%),4 (30.77%),9 (6.77%),3 (5.36%),3 (14.29%),1 (100.0%),...,2,0,1,1,0,0,0,1,0,0
CCALA_043,6569,214(3.26%),4 (6.78%),2 (1.94%),1 (33.33%),1 (7.14%),7 (5.88%),1 (1.89%),3 (13.64%),1 (100.0%),...,1,1,0,0,0,0,0,0,0,0
CCALA_051,5661,198(3.5%),4 (12.12%),3 (3.49%),1 (33.33%),1 (6.67%),4 (3.96%),2 (3.57%),2 (9.09%),1 (100.0%),...,1,0,0,0,0,0,0,0,0,0
[FACHB-1243],6077,212(3.49%),2 (5.26%),4 (4.6%),1 (25.0%),2 (13.33%),4 (3.67%),1 (1.75%),3 (13.64%),1 (100.0%),...,0,0,1,1,1,0,0,0,0,0
SAG_39.79,7547,258(3.42%),8 (10.96%),2 (2.11%),1 (33.33%),2 (13.33%),8 (6.2%),1 (1.79%),3 (13.04%),1 (100.0%),...,1,1,0,0,0,0,0,0,0,0
PCC_7203,5696,207(3.63%),2 (7.69%),1 (1.27%),1 (50.0%),2 (13.33%),6 (6.19%),2 (3.57%),3 (13.64%),1 (100.0%),...,1,1,0,0,0,0,0,0,0,0


In [56]:
#save df to csv
df.to_csv("/home/davide/Desktop/tabellafinale.csv")


In [15]:
brites_joined= {k:sum(v.values()) for k,v in brites.items()}
brites_joined
brites_motifs_joined= {k:sum(v.values()) for k,v in brites_motifs.items()}

In [16]:
file="/home/davide/Desktop/genomiChro/intergeniche_RefSeq/fimoall/joined/best_site.narrowPeak"
#gene = 
#names=[locustopid.get(line.split()[0],'') for line in open(file).readlines()]
input_file="/home/davide/Desktop/genomiChro/intergeniche_RefSeq/joined.fa"
gene_dict = {}
with open(input_file, 'r') as file:  # replace 'filename.fasta' with your actual file path
    for line in file:
        if line.startswith('>'):
            parts = line.split()
            locus_tag = parts[0][1:]
            for i,part in enumerate(parts):
                if part=="gene" and parts[i+2][0].isalpha():
                    gene_name = parts[i+2]
                    break
                else:
                    gene_name = "None"
            if locus_tag in locustopid:
                gene_dict[locustopid[locus_tag]] = gene_name
   

gene_dict

{'WP_106170542.1': 'None',
 'WP_127025321.1': 'None',
 'WP_106170550.1': 'None',
 'WP_106170552.1': 'None',
 'WP_106170557.1': 'None',
 'WP_106170560.1': 'None',
 'WP_146138844.1': 'None',
 'WP_158631969.1': 'None',
 'WP_146138845.1': 'None',
 'WP_106170568.1': 'None',
 'WP_015153616.1': 'None',
 'WP_106170565.1': 'ychF',
 'WP_106170570.1': 'None',
 'WP_015153614.1': 'None',
 'WP_015153615.1': 'smpB',
 'WP_106166432.1': 'None',
 'WP_106166433.1': 'pyk',
 'WP_106166439.1': 'None',
 'WP_106166440.1': 'bchH',
 'WP_258183830.1': 'None',
 'WP_106166442.1': 'None',
 'WP_015155204.1': 'None',
 'WP_106166431.1': 'None',
 'WP_241994169.1': 'None',
 'WP_106166434.1': 'None',
 'WP_106166435.1': 'None',
 'WP_106166436.1': 'None',
 'WP_106166437.1': 'cax',
 'WP_106166438.1': 'cax',
 'WP_241994167.1': 'None',
 'WP_106166441.1': 'None',
 'WP_199755456.1': 'None',
 'WP_158631891.1': 'None',
 'WP_106170578.1': 'None',
 'WP_106170580.1': 'None',
 'WP_158631892.1': 'None',
 'WP_015157480.1': 'None',
 'WP

In [17]:
file="/home/davide/Desktop/genomiChro/intergeniche_RefSeq/joined.fa"
from scipy.stats import hypergeom
#count number of sequences in fasta file
def count_sequences_in_fasta(file_path):
    with open(file_path, 'r') as file:
        return sum(1 for line in file if line.startswith('>'))

M = count_sequences_in_fasta(file)
n = len(names)
p_values = {}
enrichments = {}
for ko in brites_motifs_joined:
    N = brites_joined[ko]
    x = brites_motifs_joined[ko]
    enrichments[ko] = (x/n)/(N/M)
    enrichments[ko] = 'positive' if enrichments[ko]>1 else 'negative'
    p_values[ko] = hypergeom.sf(x-1, M, N, n)
    #uso distribuzione ipergeometrica per calcolare p-values
df = pd.DataFrame(list(p_values.items()), columns=['ko term', 'p-value'])
df['enrichment'] = df['ko term'].map(enrichments)


df = df.sort_values(by='p-value', ascending=True)
pd.set_option('display.max_rows', df.shape[0]+1)

In [18]:
df

,ko term,p-value,enrichment
25,ko03400 - DNA repair and recombination proteins,5.827873e-17,positive
23,ko03032 - DNA replication proteins,2.205853e-06,positive
9,ko01006 - Prenyltransferases,3.181548e-02,positive
18,ko03016 - Transfer RNA biogenesis,5.215056e-02,positive
15,ko03019 - Messenger RNA biogenesis,6.469089e-02,positive
30,ko02022 - Two-component system,8.744878e-02,positive
8,ko01008 - Polyketide biosynthesis proteins,1.499579e-01,positive
13,ko03000 - Transcription factors,1.670386e-01,positive
3,ko01002 - Peptidases and inhibitors,3.758128e-01,positive
1,ko01001 - Protein kinases,4.318219e-01,positive


In [19]:
for k,v in brites_genes.items():
    for k2,v2 in v.items():
        print(k2,v2)
        for i,el in enumerate(v2):
            if el in gene_dict and gene_dict[el]!="None":
                v2[i]=gene_dict[el]

SAG_2025 ['WP_317105586.1', 'WP_317108507.1', 'WP_317104870.1', 'WP_317104666.1', 'WP_317110507.1', 'WP_039714178.1', 'WP_317105825.1', 'WP_317105417.1', 'WP_317109189.1', 'WP_317106651.1', 'WP_015152415.1', 'WP_317105752.1', 'WP_317106964.1', 'WP_015156433.1', 'WP_317107695.1', 'WP_317110020.1', 'WP_317105483.1', 'WP_317109341.1', 'WP_015157307.1', 'WP_317110448.1', 'WP_317106042.1']
PCC_7203 ['WP_015153605.1', 'WP_015156998.1', 'WP_219336057.1', 'WP_015154828.1', 'WP_015155906.1', 'WP_015153731.1', 'WP_015153485.1', 'WP_015152861.1', 'WP_015156635.1', 'WP_219336152.1', 'WP_015152415.1', 'WP_015152349.1', 'WP_015156433.1', 'WP_015155865.1', 'WP_015155403.1', 'WP_015152734.1', 'WP_015155069.1', 'WP_015154962.1', 'WP_015156800.1', 'WP_015154849.1', 'WP_015157307.1', 'WP_015153048.1', 'WP_015153991.1']
CCNUC1 ['WP_015153605.1', 'WP_106547394.1', 'WP_250015077.1', 'WP_250017084.1', 'WP_015155906.1', 'WP_250012819.1', 'WP_250016307.1', 'WP_015153485.1', 'WP_250015484.1', 'WP_015156635.1', 

In [20]:
#set ko_term as index
df.set_index('ko term', inplace=True)
df

,p-value,enrichment
ko term,,
ko03400 - DNA repair and recombination proteins,5.827873e-17,positive
ko03032 - DNA replication proteins,2.205853e-06,positive
ko01006 - Prenyltransferases,3.181548e-02,positive
ko03016 - Transfer RNA biogenesis,5.215056e-02,positive
ko03019 - Messenger RNA biogenesis,6.469089e-02,positive
ko02022 - Two-component system,8.744878e-02,positive
ko01008 - Polyketide biosynthesis proteins,1.499579e-01,positive
ko03000 - Transcription factors,1.670386e-01,positive
ko01002 - Peptidases and inhibitors,3.758128e-01,positive


In [22]:
brites_string=deepcopy(brites_genes)
import json

brites_string= {outer_key: {inner_key: inner_value for inner_key, inner_value in outer_value.items() if inner_value != []} for outer_key, outer_value in brites_string.items()}
brites_string = {key: json.dumps(value) for key, value in brites_string.items() for key2,value2 in value.items()}
series_dict = {key: pd.Series(value) for key, value in brites_string.items()}
df_brites_string = pd.DataFrame(series_dict).T
df_brites_string
#rename con column to geni con motivo



,0
ko01000 - Enzymes,"{""SAG_2025"": [""WP_317105586.1"", ""metE"", ""miaA""..."
ko01001 - Protein kinases,"{""SAG_2025"": [""WP_317106342.1"", ""WP_317105483...."
ko01002 - Peptidases and inhibitors,"{""SAG_2025"": [""lexA""], ""PCC_7203"": [""lexA""], ""..."
ko01011 - Peptidoglycan biosynthesis and degradation proteins,"{""PCC_7203"": [""WP_015154962.1""], ""CCNUC1"": [""W..."
ko01008 - Polyketide biosynthesis proteins,"{""SAG_2025"": [""WP_317106964.1""], ""TS-821"": [""W..."
ko01006 - Prenyltransferases,"{""SAG_2025"": [""miaA""], ""PCC_7203"": [""miaA""], ""..."
ko01007 - Amino acid related enzymes,"{""SAG_2025"": [""WP_317104666.1""], ""PCC_7203"": [..."
ko00194 - Photosynthesis proteins,"{""PCC_7203"": [""psb28""], ""CCNUC1"": [""psb28""], ""..."
ko03000 - Transcription factors,"{""SAG_2025"": [""WP_317110206.1""], ""PCC_7203"": [..."
ko03019 - Messenger RNA biogenesis,"{""SAG_2025"": [""ppk2""], ""PCC_7203"": [""ppk2""], ""..."


In [45]:
for k,v in pathways_genes.items():
    for k2,v2 in v.items():
        for i,el in enumerate(v2):
            if el in gene_dict and gene_dict[el]!="None":
                v2[i]=gene_dict[el]
pathways_string=deepcopy(pathways_genes)
pathways_string= {outer_key: {inner_key: inner_value for inner_key, inner_value in outer_value.items() if inner_value != []} for outer_key, outer_value in pathways_string.items()}
pathways_string = {key: json.dumps(value) for key, value in pathways_string.items() for key2,value2 in value.items()}
series_dict = {key: pd.Series(value) for key, value in pathways_string.items()}
df_pathways_string = pd.DataFrame(series_dict).T
df_pathways_string

,0
Metabolic pathways,"{""SAG_2025"": [""WP_317106986.1"", ""metE"", ""glgP""..."
Biosynthesis of secondary metabolites,"{""SAG_2025"": [""WP_317106986.1"", ""metE"", ""glgP""..."
Microbial metabolism in diverse environments,"{""SAG_2025"": [""WP_317106986.1"", ""WP_106218380...."
Carbon metabolism,"{""SAG_2025"": [""WP_106218380.1"", ""WP_317108570...."
2-Oxocarboxylic acid metabolism,"{""SAG_2025"": [""WP_317104666.1"", ""argB""], ""PCC_..."
...,...
Insulin resistance,"{""SAG_2025"": [""glgP""]}"
beta-Lactam resistance,"{""TS-821"": [""WP_246841293.1"", ""WP_104548237.1""]}"
Cationic antimicrobial peptide (CAMP) resistance,"{""SAG_2025"": [""WP_317109188.1""], ""PCC_7203"": [..."
Platinum drug resistance,"{""PCC_7203"": [""WP_015156874.1""], ""SAG_39.79"": ..."


In [30]:
pathways

{'Metabolic pathways': {'SAG_2025': 1037,
  'PCC_7203': 951,
  'CCNUC1': 981,
  'CCMEE_29': 884,
  'TS-821': 844,
  'SAG_39.79': 1035,
  'CCALA_043': 996,
  'CCALA_051': 957,
  '[FACHB-1243]': 919},
 'Biosynthesis of secondary metabolites': {'SAG_2025': 425,
  'PCC_7203': 397,
  'CCNUC1': 409,
  'CCMEE_29': 398,
  'TS-821': 349,
  'SAG_39.79': 428,
  'CCALA_043': 421,
  'CCALA_051': 393,
  '[FACHB-1243]': 370},
 'Microbial metabolism in diverse environments': {'SAG_2025': 228,
  'PCC_7203': 192,
  'CCNUC1': 204,
  'CCMEE_29': 183,
  'TS-821': 185,
  'SAG_39.79': 231,
  'CCALA_043': 222,
  'CCALA_051': 189,
  '[FACHB-1243]': 189},
 'Carbon metabolism': {'SAG_2025': 109,
  'PCC_7203': 98,
  'CCNUC1': 101,
  'CCMEE_29': 98,
  'TS-821': 93,
  'SAG_39.79': 108,
  'CCALA_043': 104,
  'CCALA_051': 97,
  '[FACHB-1243]': 94},
 '2-Oxocarboxylic acid metabolism': {'SAG_2025': 35,
  'PCC_7203': 33,
  'CCNUC1': 33,
  'CCMEE_29': 36,
  'TS-821': 30,
  'SAG_39.79': 39,
  'CCALA_043': 33,
  'CCALA_051

In [52]:
file = "/home/davide/Desktop/genomiChro/intergeniche_RefSeq/joined.fa"
from scipy.stats import hypergeom


#count number of sequences in fasta file
def count_sequences_in_fasta(file_path):
    with open(file_path, 'r') as file:
        return sum(1 for line in file if line.startswith('>'))

pathways_joined= {k:sum(v.values()) for k,v in pathways.items()}
pathways_motifs_joined= {k:sum(v.values()) for k,v in pathways_motifs.items()}
M = count_sequences_in_fasta(file)
n = len(names)
p_values = {}
enrichments = {}
for ko in pathways_motifs_joined:
    N = pathways_joined[ko]
    x = pathways_motifs_joined[ko]
    enrichments[ko] = (x / n) / (N / M)
    enrichments[ko] = 'positive' if enrichments[ko] > 1 else 'negative'
    p_values[ko] = hypergeom.sf(x - 1, M, N, n)
    #uso distribuzione ipergeometrica per calcolare p-values
df_pathway = pd.DataFrame(list(p_values.items()), columns=['pathway', 'p-value'])
df_pathway['enrichment'] = df_pathway['pathway'].map(enrichments)
df_pathway.sort_values(by='p-value', ascending=True,inplace=True)
pd.set_option('display.max_rows', df.shape[0] + 1)


In [53]:
df_pathway.set_index("pathway",inplace=True)
df_pathway

,p-value,enrichment
pathway,,
Mismatch repair,7.744156e-17,positive
Homologous recombination,2.961694e-12,positive
DNA replication,2.632612e-10,positive
Zeatin biosynthesis,5.183101e-06,positive
Nucleotide excision repair,5.835690e-06,positive
...,...,...
Endocytosis,1.000000e+00,negative
Lysosome,1.000000e+00,negative
Biosynthesis of type II polyketide products,1.000000e+00,negative


In [54]:

df_pathway=df_pathway.join(df_pathways_string)
df_pathway.rename(columns={0:"geni con motivo"},inplace=True)
#df_pathway["p-value"]=df_pathway["p-value"].round(2)
df_pathway



,p-value,enrichment,geni con motivo
pathway,,,
Mismatch repair,7.744156e-17,positive,"{""SAG_2025"": [""ligA"", ""WP_317108552.1"", ""WP_31..."
Homologous recombination,2.961694e-12,positive,"{""SAG_2025"": [""WP_317108552.1"", ""WP_317109348...."
DNA replication,2.632612e-10,positive,"{""SAG_2025"": [""ligA"", ""WP_317108552.1"", ""WP_31..."
Zeatin biosynthesis,5.183101e-06,positive,"{""SAG_2025"": [""miaA""], ""PCC_7203"": [""miaA""], ""..."
Nucleotide excision repair,5.835690e-06,positive,"{""SAG_2025"": [""ligA"", ""uvrB""], ""PCC_7203"": [""l..."
...,...,...,...
Endocytosis,1.000000e+00,negative,NaN
Lysosome,1.000000e+00,negative,NaN
Biosynthesis of type II polyketide products,1.000000e+00,negative,NaN


In [55]:
def round_pvalues(df):
    # Round p-values column while preserving exponential notation
    df['p-value'] = df['p-value'].apply(lambda x: '{:.2e}'.format(x))
round_pvalues(df_pathway)
df_pathway

,p-value,enrichment,geni con motivo
pathway,,,
Mismatch repair,7.74e-17,positive,"{""SAG_2025"": [""ligA"", ""WP_317108552.1"", ""WP_31..."
Homologous recombination,2.96e-12,positive,"{""SAG_2025"": [""WP_317108552.1"", ""WP_317109348...."
DNA replication,2.63e-10,positive,"{""SAG_2025"": [""ligA"", ""WP_317108552.1"", ""WP_31..."
Zeatin biosynthesis,5.18e-06,positive,"{""SAG_2025"": [""miaA""], ""PCC_7203"": [""miaA""], ""..."
Nucleotide excision repair,5.84e-06,positive,"{""SAG_2025"": [""ligA"", ""uvrB""], ""PCC_7203"": [""l..."
...,...,...,...
Endocytosis,1.00e+00,negative,NaN
Lysosome,1.00e+00,negative,NaN
Biosynthesis of type II polyketide products,1.00e+00,negative,NaN


In [56]:
df_pathway.to_csv("/home/davide/Desktop/ArricchimentoKEGGpathways.csv")

In [137]:
#rename column 0
df_brites_string.columns=["geni con motivo"]
df_brites_string

,geni con motivo
ko01000 - Enzymes,"{""SAG_2025"": [""WP_317105586.1"", ""metE"", ""miaA""..."
ko01001 - Protein kinases,"{""SAG_2025"": [""WP_317106342.1"", ""WP_317105483...."
ko01002 - Peptidases and inhibitors,"{""SAG_2025"": [""lexA""], ""PCC_7203"": [""lexA""], ""..."
ko01011 - Peptidoglycan biosynthesis and degradation proteins,"{""PCC_7203"": [""WP_015154962.1""], ""CCNUC1"": [""W..."
ko01008 - Polyketide biosynthesis proteins,"{""SAG_2025"": [""WP_317106964.1""], ""TS-821"": [""W..."
ko01006 - Prenyltransferases,"{""SAG_2025"": [""miaA""], ""PCC_7203"": [""miaA""], ""..."
ko01007 - Amino acid related enzymes,"{""SAG_2025"": [""WP_317104666.1""], ""PCC_7203"": [..."
ko00194 - Photosynthesis proteins,"{""PCC_7203"": [""psb28""], ""CCNUC1"": [""psb28""], ""..."
ko03000 - Transcription factors,"{""SAG_2025"": [""WP_317110206.1""], ""PCC_7203"": [..."
ko03019 - Messenger RNA biogenesis,"{""SAG_2025"": [""ppk2""], ""PCC_7203"": [""ppk2""], ""..."


In [138]:
df=df.join(df_brites_string)

In [140]:
df
#drop genes column
df.drop(columns=["genes"],inplace=True)

In [141]:
df

,p-value,enrichment,geni con motivo
ko term,,,
ko03400 - DNA repair and recombination proteins,5.827873e-17,positive,"{""SAG_2025"": [""lexA"", ""ligA"", ""gyrB"", ""recA"", ..."
ko03032 - DNA replication proteins,2.205853e-06,positive,"{""SAG_2025"": [""ligA"", ""gyrB""], ""PCC_7203"": [""l..."
ko01006 - Prenyltransferases,3.181548e-02,positive,"{""SAG_2025"": [""miaA""], ""PCC_7203"": [""miaA""], ""..."
ko03016 - Transfer RNA biogenesis,5.215056e-02,positive,"{""SAG_2025"": [""miaA"", ""mnmG"", ""queA""], ""PCC_72..."
ko03019 - Messenger RNA biogenesis,6.469089e-02,positive,"{""SAG_2025"": [""ppk2""], ""PCC_7203"": [""ppk2""], ""..."
ko02022 - Two-component system,8.744878e-02,positive,"{""SAG_2025"": [""WP_317106342.1"", ""WP_317105434...."
ko01008 - Polyketide biosynthesis proteins,1.499579e-01,positive,"{""SAG_2025"": [""WP_317106964.1""], ""TS-821"": [""W..."
ko03000 - Transcription factors,1.670386e-01,positive,"{""SAG_2025"": [""WP_317110206.1""], ""PCC_7203"": [..."
ko01002 - Peptidases and inhibitors,3.758128e-01,positive,"{""SAG_2025"": [""lexA""], ""PCC_7203"": [""lexA""], ""..."


In [142]:
#save df to csv
df.to_csv("/home/davide/Desktop/arricchimentoKOBrite.csv")

In [223]:
file="/home/davide/Desktop/genomiChro/intergeniche_RefSeq/fimo/Chroococcidiopsis_sp._CCMEE_29_GCF_023558375_intergen/fimo.tsv"
motiviCCMEE=pd.read_csv(file,sep="\t",header=None,skipfooter=4,engine='python')
#use onlu column 0 and 3
motiviCCMEE=motiviCCMEE[[2,9]]
motiviCCMEE
#riname columns
motiviCCMEE.columns=["locus","motivo"]

In [224]:
#now map the locus column with the protein_id
motiviCCMEE["protein_id"]=motiviCCMEE["locus"].map(locustopid)
motiviCCMEE
#now map with gene_dict
motiviCCMEE["gene"]=motiviCCMEE["protein_id"].map(gene_dict)
motiviCCMEE
#set locus as index
motiviCCMEE.set_index("locus",inplace=True)
motiviCCMEE
#reorder columns
motiviCCMEE=motiviCCMEE[["protein_id","gene","motivo"]]


In [225]:
#drop rows with protein_id NaN
motiviCCMEE.dropna(subset=["protein_id"],inplace=True)
motiviCCMEE
#set protein_id as index
motiviCCMEE.set_index("protein_id",inplace=True)

In [226]:
motiviCCMEE=motiviCCMEE[["gene","motivo"]]
motiviCCMEE

,gene,motivo
protein_id,,
WP_250124683.1,None,TAGTACATTTGTACTA
WP_250121527.1,None,TAGTACAAATGTACTA
WP_250121243.1,None,TAGTACATATGTACTA
WP_250121242.1,None,TAGTACATATGTACTA
WP_250121656.1,None,TAGTAAATTTGTACTA
...,...,...
WP_250121953.1,None,CATTACAACCTTACTG
WP_250124124.1,None,AAGTGCATTTGCTATT
WP_250124279.1,None,TAGAAAAGCTGGTCTA


In [227]:
motiviCCMEE

,gene,motivo
protein_id,,
WP_250124683.1,None,TAGTACATTTGTACTA
WP_250121527.1,None,TAGTACAAATGTACTA
WP_250121243.1,None,TAGTACATATGTACTA
WP_250121242.1,None,TAGTACATATGTACTA
WP_250121656.1,None,TAGTAAATTTGTACTA
...,...,...
WP_250121953.1,None,CATTACAACCTTACTG
WP_250124124.1,None,AAGTGCATTTGCTATT
WP_250124279.1,None,TAGAAAAGCTGGTCTA


In [228]:
proteineCCMEE="/home/davide/Desktop/genomiChro/annotati_Refseq/Chroococcidiopsis_sp._CCMEE_29_(cyanobacteria)_GCF_023558375.gbff"
#use parse biopython to find cds start and finish for every protein with protein_id in motiviCCMEE datafram
from Bio import SeqIO

genoma = SeqIO.parse(proteineCCMEE, "genbank")
for seq in genoma:
    for record in seq.features:
        if record.type == "CDS":
            if "protein_id" in record.qualifiers:
                protein_id = record.qualifiers["protein_id"][0]
                if protein_id in motiviCCMEE.index:
                    motiviCCMEE.loc[protein_id,"location"] = str(record.location)
                    if "gene" in record.qualifiers:
                        print(record.qualifiers["gene"])
motiviCCMEE
#order ge

['fetB']
['radA']
['lpdA']
['crtH']
['upp']
['ggt']
['rplC']
['psbD']
['psbD']
['lexA']
['cysS']
['ftsH']
['xseA']
['recA']
['def']
['bioA']
['thiD']
['topA']
['sepF']
['rpsT']
['hisD']
['eboE']


,gene,motivo,location
protein_id,,,
WP_250124683.1,None,TAGTACATTTGTACTA,[1199937:1200360](-)
WP_250121527.1,None,TAGTACAAATGTACTA,[3293848:3294115](-)
WP_250121243.1,None,TAGTACATATGTACTA,[2963734:2964394](+)
WP_250121242.1,None,TAGTACATATGTACTA,[2961959:2963384](-)
WP_250121656.1,None,TAGTAAATTTGTACTA,[3439950:3442179](-)
...,...,...,...
WP_250121953.1,None,CATTACAACCTTACTG,[3787202:3788060](+)
WP_250124124.1,None,AAGTGCATTTGCTATT,[580890:581088](-)
WP_250124279.1,None,TAGAAAAGCTGGTCTA,[745640:746276](-)


In [229]:
#order genes by name
motiviCCMEE.sort_values(by="gene",inplace=True,ascending=False)
motiviCCMEE

,gene,motivo,location
protein_id,,,
WP_250121492.1,xseA,TAGTGCAGATATACTA,[3252216:3253449](-)
WP_250124546.1,upp,GATTAAAAATGTTCTA,[1045043:1045694](+)
WP_250123153.1,topA,CAGAGCAATCTTACTC,[5167682:5170382](+)
WP_250122239.1,thiD,TAGAATAGATGCAATT,[4115914:4116733](-)
WP_250123172.1,sepF,TAGAATAGCTGAACTT,[5199317:5199749](+)
...,...,...,...
WP_250125818.1,None,TAGTGCAAACAATCTA,[2495981:2496932](-)
WP_250125469.1,None,TAGTGTAGTTTTACCA,[2084973:2085318](+)
WP_250124778.1,None,TAGTGCAATTGTACAA,[1319906:1320623](-)


In [230]:
motiviCCMEE.to_csv("/home/davide/Desktop/motiviCCMEE.csv")

In [231]:
file="/home/davide/Desktop/annotazioniGOkoCCMEE29.csv"
df=pd.read_csv(file)
df
#drop gene column
df.drop(columns=["gene"],inplace=True)
df
#remove duplicates
df.drop_duplicates(inplace=True)
df
#drop rows with same protein_id only first
df.drop_duplicates(subset=["protein_id"],keep="first",inplace=True)
df

,protein_id,GO_process,GO_component,GO_function,GOs_eggnog,GOs_interpro,ko_eggnog,ko_blastkoala,GO_process_pannzer2,KEGG_brite,KEGG_pathways
0,WP_250124752.1,['GO:0006098 - pentose-phosphate shunt [Eviden...,NaN,['GO:0004345 - glucose-6-phosphate dehydrogena...,-,GO:0004345(PANTHER)|GO:0005829(PANTHER)|GO:000...,ko:K00036,K00036,GO:0006098 - pentose-phosphate shunt,ko04147 Exosome,"01100 Metabolic pathways, 01110 Biosynthesis o..."
2,WP_250123417.1,NaN,NaN,NaN,-,GO:0016491(PANTHER),ko:K00514,K00514,GO:0016117 - carotenoid biosynthetic process,NaN,"01100 Metabolic pathways, 01110 Biosynthesis o..."
3,WP_250121918.1,NaN,NaN,NaN,-,-,-,NaN,NaN,NaN,NaN
4,WP_250124094.1,['GO:0042254 - ribosome biogenesis [Evidence I...,['GO:0005737 - cytoplasm [Evidence IEA]'],['GO:0003924 - GTPase activity [Evidence IEA]'...,-,-,ko:K14540,K14540,NaN,ko03009 Ribosome biogenesis,NaN
5,WP_250123747.1,NaN,NaN,NaN,-,-,-,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
6661,WP_250123219.1,NaN,NaN,NaN,-,-,-,NaN,NaN,NaN,NaN
6662,WP_250123218.1,NaN,NaN,NaN,-,GO:0003824(InterPro)|GO:0051536(InterPro),-,NaN,NaN,NaN,NaN
6663,WP_250123216.1,NaN,NaN,NaN,-,GO:0005887(PANTHER),ko:K03589,K03589,NaN,ko03036 Chromosome and associated proteins,04112 Cell cycle - Caulobacter
6664,WP_275983398.1,NaN,NaN,NaN,"GO:0005575,GO:0005576",NaN,ko:K14645,NaN,NaN,NaN,NaN


In [232]:
#set index protein_id
df.set_index("protein_id",inplace=True)
#join motiviCCMEE with df
motiviCCMEE=motiviCCMEE.join(df,how="inner")
motiviCCMEE

,gene,motivo,location,GO_process,GO_component,GO_function,GOs_eggnog,GOs_interpro,ko_eggnog,ko_blastkoala,GO_process_pannzer2,KEGG_brite,KEGG_pathways
protein_id,,,,,,,,,,,,,
WP_250121492.1,xseA,TAGTGCAGATATACTA,[3252216:3253449](-),['GO:0006308 - DNA catabolic process [Evidence...,['GO:0009318 - exodeoxyribonuclease VII comple...,['GO:0008855 - exodeoxyribonuclease VII activi...,-,GO:0003676(InterPro),ko:K03601,K03601,NaN,ko03400 DNA repair and recombination proteins,03430 Mismatch repair
WP_250124546.1,upp,GATTAAAAATGTTCTA,[1045043:1045694](+),['GO:0009116 - nucleoside metabolic process [E...,NaN,['GO:0004845 - uracil phosphoribosyltransferas...,"GO:0003674,GO:0003824,GO:0004849,GO:0005575,GO...",GO:0004845(InterPro)|GO:0006223(InterPro),ko:K00761,K00761,GO:0006223 - uracil salvage,NaN,"01100 Metabolic pathways, 01232 Nucleotide met..."
WP_250123153.1,topA,CAGAGCAATCTTACTC,[5167682:5170382](+),['GO:0006260 - DNA replication [Evidence IEA]'...,['GO:0005737 - cytoplasm [Evidence IEA]'],"['GO:0003677 - DNA binding [Evidence IEA]', 'G...",-,-,ko:K03168,K03168,GO:0006265 - DNA topological change,"ko03032 DNA replication proteins, ko03400 DNA ...",NaN
WP_250122239.1,thiD,TAGAATAGATGCAATT,[4115914:4116733](-),['GO:0009228 - thiamine biosynthetic process [...,NaN,['GO:0008902 - hydroxymethylpyrimidine kinase ...,-,-,ko:K00941,K00941,NaN,NaN,"01100 Metabolic pathways, 01240 Biosynthesis o..."
WP_250123172.1,sepF,TAGAATAGCTGAACTT,[5199317:5199749](+),['GO:0090529 - cell septum assembly [Evidence ...,NaN,NaN,-,GO:0090529(InterPro),ko:K09772,K09772,NaN,ko03036 Chromosome and associated proteins,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
WP_250125818.1,None,TAGTGCAAACAATCTA,[2495981:2496932](-),NaN,NaN,NaN,-,GO:0016740(InterPro),-,NaN,NaN,NaN,NaN
WP_250125469.1,None,TAGTGTAGTTTTACCA,[2084973:2085318](+),NaN,NaN,NaN,-,-,ko:K09940,K09940,NaN,NaN,NaN
WP_250124778.1,None,TAGTGCAATTGTACAA,[1319906:1320623](-),NaN,NaN,NaN,-,-,-,NaN,NaN,NaN,NaN


In [237]:
#order by KEGG_brite by number of values
motiviCCMEE= motiviCCMEE.sort_values(by='KEGG_brite', ascending=False, na_position='last')
motiviCCMEE
df=motiviCCMEE
# Assuming 'column_name' is the name of the column you want to sort by
df['cluster_size'] = df.groupby('KEGG_brite')['KEGG_brite'].transform('size')

# Sort the DataFrame by the 'cluster_size' column in descending order
df = df.sort_values(by='cluster_size', ascending=False)



# Drop the 'cluster_size' column if it's no longer needed
df = df.drop(columns='cluster_size')
df
#put KEGG_brite as first column
df = df[["KEGG_brite"]+[col for col in df.columns if col not in ["KEGG_brite"]]]
df

,KEGG_brite,gene,motivo,location,GO_process,GO_component,GO_function,GOs_eggnog,GOs_interpro,ko_eggnog,ko_blastkoala,GO_process_pannzer2,KEGG_pathways
protein_id,,,,,,,,,,,,,
WP_250121496.1,ko03400 DNA repair and recombination proteins,None,CAGTAAAACTTCACTC,[3256115:3257708](-),NaN,NaN,NaN,-,NaN,ko:K03722,K03722,NaN,NaN
WP_250121493.1,ko03400 DNA repair and recombination proteins,recA,TAGTATATCTGCACTA,[3253689:3254766](+),"['GO:0006281 - DNA repair [Evidence IEA]', 'GO...",['GO:0005737 - cytoplasm [Evidence IEA]'],"['GO:0003677 - DNA binding [Evidence IEA]', 'G...","GO:0005575,GO:0005622,GO:0005623,GO:0005737,GO...",GO:0003697(InterPro)|GO:0005524(InterPro)|GO:0...,ko:K03553,K03553,GO:0006310 - DNA recombination,03440 Homologous recombination
WP_250121656.1,ko03400 DNA repair and recombination proteins,None,TAGTAAATTTGTACTA,[3439950:3442179](-),NaN,NaN,['GO:0004386 - helicase activity [Evidence IEA]'],-,GO:0006310(PANTHER)|GO:0009338(PANTHER)|GO:001...,ko:K03581,K03581,GO:0032508 - DNA duplex unwinding,03440 Homologous recombination
WP_250123666.1,ko03400 DNA repair and recombination proteins,None,CAGTGCAAATGTACTG,[78399:81426](+),NaN,NaN,NaN,-,GO:0006302(InterPro)|GO:0016887(InterPro),ko:K03546,K03546,GO:0006302 - double-strand break repair,NaN
WP_250122036.1,ko03400 DNA repair and recombination proteins,None,TAGTACATTGTTACTA,[3874281:3875715](+),NaN,NaN,NaN,-,-,ko:K01669,K01669,GO:0006139 - nucleobase-containing compound me...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
WP_250124389.1,NaN,None,TAGTAAAATCTTAATC,[870964:872602](-),NaN,NaN,['GO:0000287 - magnesium ion binding [Evidence...,-,-,ko:K01652,K01652,NaN,"01100 Metabolic pathways, 01110 Biosynthesis o..."
WP_250125818.1,NaN,None,TAGTGCAAACAATCTA,[2495981:2496932](-),NaN,NaN,NaN,-,GO:0016740(InterPro),-,NaN,NaN,NaN
WP_250125469.1,NaN,None,TAGTGTAGTTTTACCA,[2084973:2085318](+),NaN,NaN,NaN,-,-,ko:K09940,K09940,NaN,NaN


In [238]:
motiviCCMEE.to_csv("/home/davide/Desktop/motiviCCMEE.csv")

In [18]:
file="/home/davide/Downloads/motiviCCMEE29_google.csv"
import pandas as pd
df=pd.read_csv(file)
df.set_index("protein_id",inplace=True)
df

,Unnamed: 0,gene,motivo,location,KEGG_brite,GO_process,GO_component,GO_function,GOs_eggnog,GOs_interpro,ko_eggnog,ko_blastkoala,GO_process_pannzer2,KEGG_pathways,cluster_size
protein_id,,,,,,,,,,,,,,,
NaN,Riparo del DNA e ricombinazione,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WP_250121492.1,NaN,xseA,TAGTGCAGATATACTA,[3252216:3253449](-),ko03400 DNA repair and recombination proteins,['GO:0006308 - DNA catabolic process [Evidence...,['GO:0009318 - exodeoxyribonuclease VII comple...,['GO:0008855 - exodeoxyribonuclease VII activi...,-,GO:0003676(InterPro),ko:K03601,K03601,NaN,03430 Mismatch repair,7.0
WP_250121496.1,NaN,NaN,CAGTAAAACTTCACTC,[3256115:3257708](-),ko03400 DNA repair and recombination proteins,NaN,NaN,NaN,-,NaN,ko:K03722,K03722,NaN,NaN,7.0
WP_250121493.1,NaN,recA,TAGTATATCTGCACTA,[3253689:3254766](+),ko03400 DNA repair and recombination proteins,"['GO:0006281 - DNA repair [Evidence IEA]', 'GO...",['GO:0005737 - cytoplasm [Evidence IEA]'],"['GO:0003677 - DNA binding [Evidence IEA]', 'G...","GO:0005575,GO:0005622,GO:0005623,GO:0005737,GO...",GO:0003697(InterPro)|GO:0005524(InterPro)|GO:0...,ko:K03553,K03553,GO:0006310 - DNA recombination,03440 Homologous recombination,7.0
WP_250121656.1,NaN,NaN,TAGTAAATTTGTACTA,[3439950:3442179](-),ko03400 DNA repair and recombination proteins,NaN,NaN,['GO:0004386 - helicase activity [Evidence IEA]'],-,GO:0006310(PANTHER)|GO:0009338(PANTHER)|GO:001...,ko:K03581,K03581,GO:0032508 - DNA duplex unwinding,03440 Homologous recombination,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WP_250124389.1,NaN,NaN,TAGTAAAATCTTAATC,[870964:872602](-),NaN,NaN,NaN,['GO:0000287 - magnesium ion binding [Evidence...,-,-,ko:K01652,K01652,NaN,"01100 Metabolic pathways, 01110 Biosynthesis o...",NaN
WP_250125818.1,NaN,NaN,TAGTGCAAACAATCTA,[2495981:2496932](-),NaN,NaN,NaN,NaN,-,GO:0016740(InterPro),-,NaN,NaN,NaN,NaN
WP_250125469.1,NaN,NaN,TAGTGTAGTTTTACCA,[2084973:2085318](+),NaN,NaN,NaN,NaN,-,-,ko:K09940,K09940,NaN,NaN,NaN


In [19]:
file_genoma="/home/davide/Desktop/genomiChro/annotati_Refseq/Chroococcidiopsis_sp._CCMEE_29_(cyanobacteria)_GCF_023558375.gbff"
from Bio import SeqIO


genoma = SeqIO.parse(file_genoma, "genbank")
for seq in genoma:
    for record in seq.features:
        if record.type == "CDS":
            if "protein_id" in record.qualifiers:
                protein_id = record.qualifiers["protein_id"][0]
                if protein_id in df.index and record.qualifiers["product"][0]!="hypothetical protein" :
                    df.loc[protein_id,"product"] = record.qualifiers["product"][0]


In [22]:
df

,Unnamed: 0,gene,motivo,location,KEGG_brite,GO_process,GO_component,GO_function,GOs_eggnog,GOs_interpro,ko_eggnog,ko_blastkoala,GO_process_pannzer2,KEGG_pathways,cluster_size,product
protein_id,,,,,,,,,,,,,,,,
NaN,Riparo del DNA e ricombinazione,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WP_250121492.1,NaN,xseA,TAGTGCAGATATACTA,[3252216:3253449](-),ko03400 DNA repair and recombination proteins,['GO:0006308 - DNA catabolic process [Evidence...,['GO:0009318 - exodeoxyribonuclease VII comple...,['GO:0008855 - exodeoxyribonuclease VII activi...,-,GO:0003676(InterPro),ko:K03601,K03601,NaN,03430 Mismatch repair,7.0,exodeoxyribonuclease VII large subunit
WP_250121496.1,NaN,NaN,CAGTAAAACTTCACTC,[3256115:3257708](-),ko03400 DNA repair and recombination proteins,NaN,NaN,NaN,-,NaN,ko:K03722,K03722,NaN,NaN,7.0,ATP-dependent DNA helicase
WP_250121493.1,NaN,recA,TAGTATATCTGCACTA,[3253689:3254766](+),ko03400 DNA repair and recombination proteins,"['GO:0006281 - DNA repair [Evidence IEA]', 'GO...",['GO:0005737 - cytoplasm [Evidence IEA]'],"['GO:0003677 - DNA binding [Evidence IEA]', 'G...","GO:0005575,GO:0005622,GO:0005623,GO:0005737,GO...",GO:0003697(InterPro)|GO:0005524(InterPro)|GO:0...,ko:K03553,K03553,GO:0006310 - DNA recombination,03440 Homologous recombination,7.0,recombinase RecA
WP_250121656.1,NaN,NaN,TAGTAAATTTGTACTA,[3439950:3442179](-),ko03400 DNA repair and recombination proteins,NaN,NaN,['GO:0004386 - helicase activity [Evidence IEA]'],-,GO:0006310(PANTHER)|GO:0009338(PANTHER)|GO:001...,ko:K03581,K03581,GO:0032508 - DNA duplex unwinding,03440 Homologous recombination,7.0,ATP-dependent RecD-like DNA helicase
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WP_250124389.1,NaN,NaN,TAGTAAAATCTTAATC,[870964:872602](-),NaN,NaN,NaN,['GO:0000287 - magnesium ion binding [Evidence...,-,-,ko:K01652,K01652,NaN,"01100 Metabolic pathways, 01110 Biosynthesis o...",NaN,acetolactate synthase large subunit
WP_250125818.1,NaN,NaN,TAGTGCAAACAATCTA,[2495981:2496932](-),NaN,NaN,NaN,NaN,-,GO:0016740(InterPro),-,NaN,NaN,NaN,NaN,"L,D-transpeptidase"
WP_250125469.1,NaN,NaN,TAGTGTAGTTTTACCA,[2084973:2085318](+),NaN,NaN,NaN,NaN,-,-,ko:K09940,K09940,NaN,NaN,NaN,DUF4870 domain-containing protein


In [23]:
df.to_csv("/home/davide/Desktop/motiviCCMEE29_google.csv")

In [48]:
file="/home/davide/Downloads/motiviCCMEE29_google.csv"
file_genoma = "/home/davide/Desktop/genomiChro/annotati_Refseq/Chroococcidiopsis_sp._CCMEE_29_(cyanobacteria)_GCF_023558375.gbff"
intergen="/home/davide/Desktop/genomiChro/intergeniche_RefSeq/Chroococcidiopsis_sp._CCMEE_29_GCF_023558375_intergen.fasta"
import pandas as pd
import re
df=pd.read_csv(file)
df.set_index("protein_id",inplace=True)
motivi=['.AGT.{4,11}ACT.', 'TAGT.{5,11}CTA.']
def cerca_motivo(motivo,seq):
    matches = [m.group() for m in re.finditer(motivo, str(seq.seq))]
    return matches
    
    
genoma = SeqIO.parse(file_genoma, "genbank")

for seq in genoma:
    for record in seq.features:
        if record.type == "CDS":
            if "protein_id" in record.qualifiers:
                protein_id = record.qualifiers["protein_id"][0]
                locus_tag = record.qualifiers["locus_tag"][0]
                fasta=SeqIO.parse(intergen, "fasta")
                #print(locus_tag)
                if protein_id in df.index:
                    for seq in fasta:
                        if str(seq.id)==str(locus_tag):
                            trova=cerca_motivo(motivi[0],seq)
                            df.loc[protein_id,"motivo .AGT.{4,11}ACT."]=trova[0] if len(trova)>0 else "None"
                            trova=cerca_motivo(motivi[1],seq)
                            df.loc[protein_id,"motivo TAGT.{5,11}CTA."]=trova[0] if len(trova)>0 else "None"
df
        




,Unnamed: 0,gene,motivo,location,KEGG_brite,GO_process,GO_component,GO_function,GOs_eggnog,GOs_interpro,ko_eggnog,ko_blastkoala,GO_process_pannzer2,KEGG_pathways,cluster_size,"motivo .AGT.{4,11}ACT.","motivo TAGT.{5,11}CTA."
protein_id,,,,,,,,,,,,,,,,,
NaN,Riparo del DNA e ricombinazione,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WP_250121492.1,NaN,xseA,TAGTGCAGATATACTA,[3252216:3253449](-),ko03400 DNA repair and recombination proteins,['GO:0006308 - DNA catabolic process [Evidence...,['GO:0009318 - exodeoxyribonuclease VII comple...,['GO:0008855 - exodeoxyribonuclease VII activi...,-,GO:0003676(InterPro),ko:K03601,K03601,NaN,03430 Mismatch repair,7.0,GAGTCTAGACTA,TAGTGCAGATATACTAT
WP_250121496.1,NaN,NaN,CAGTAAAACTTCACTC,[3256115:3257708](-),ko03400 DNA repair and recombination proteins,NaN,NaN,NaN,-,NaN,ko:K03722,K03722,NaN,NaN,7.0,CAGTAAAACTTCACTC,None
WP_250121493.1,NaN,recA,TAGTATATCTGCACTA,[3253689:3254766](+),ko03400 DNA repair and recombination proteins,"['GO:0006281 - DNA repair [Evidence IEA]', 'GO...",['GO:0005737 - cytoplasm [Evidence IEA]'],"['GO:0003677 - DNA binding [Evidence IEA]', 'G...","GO:0005575,GO:0005622,GO:0005623,GO:0005737,GO...",GO:0003697(InterPro)|GO:0005524(InterPro)|GO:0...,ko:K03553,K03553,GO:0006310 - DNA recombination,03440 Homologous recombination,7.0,TAGTATATCTGCACTA,TAGTATATCTGCACTAG
WP_250121656.1,NaN,NaN,TAGTAAATTTGTACTA,[3439950:3442179](-),ko03400 DNA repair and recombination proteins,NaN,NaN,['GO:0004386 - helicase activity [Evidence IEA]'],-,GO:0006310(PANTHER)|GO:0009338(PANTHER)|GO:001...,ko:K03581,K03581,GO:0032508 - DNA duplex unwinding,03440 Homologous recombination,7.0,TAGTAAATTTGTACTA,TAGTAAATTTGTACTAA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WP_250124389.1,NaN,NaN,TAGTAAAATCTTAATC,[870964:872602](-),NaN,NaN,NaN,['GO:0000287 - magnesium ion binding [Evidence...,-,-,ko:K01652,K01652,NaN,"01100 Metabolic pathways, 01110 Biosynthesis o...",NaN,None,None
WP_250125818.1,NaN,NaN,TAGTGCAAACAATCTA,[2495981:2496932](-),NaN,NaN,NaN,NaN,-,GO:0016740(InterPro),-,NaN,NaN,NaN,NaN,None,TAGTGCAAACAATCTAG
WP_250125469.1,NaN,NaN,TAGTGTAGTTTTACCA,[2084973:2085318](+),NaN,NaN,NaN,NaN,-,-,ko:K09940,K09940,NaN,NaN,NaN,None,None


In [49]:
df.to_csv("/home/davide/Desktop/motiviCCMEE29conricerca.csv")